In [56]:
# from data_preprocessing import preprocess_data
# import numpy as np
# import pandas as pd
# from tensorflow.keras.layers import Input, Embedding, Dense, Concatenate
# from tensorflow.keras.models import Model
# import tensorflow as tf
# 
# # Load the data
# files = [
#         "../My_datasets/2012_13.xlsx",
#         "../My_datasets/2013_14.xlsx",
#         "../My_datasets/2014_15.xlsx",
#         "../My_datasets/2015_16.xlsx",
#         "../My_datasets/2016_17.xlsx",
#         "../My_datasets/2017_18.xlsx",
#         "../My_datasets/2018_19.xlsx"
# 
# 
#     ]
# datasets = [pd.read_excel(file) for file in files]
# 
#     # Preprocess datasets
# numerical_cols_data = datasets[0].select_dtypes(include=['int64', 'float64']).columns.tolist()
# numerical_cols_data = [col for col in numerical_cols_data if col != "team identifier"]
# categorical_cols_data = datasets[0].select_dtypes(include=['object']).columns.tolist()
# categorical_cols_data = [col for col in categorical_cols_data if col != "Team"]
# preprocessed_datasets = [preprocess_data(df, numerical_cols_data, categorical_cols_data) for df in datasets]
# 
#     # Prepare test data
# test_data = preprocessed_datasets[-1][["Team", "team identifier"]].copy()
# test_data["Rank"] = test_data.index + 1
# 
#     # Aggregate features by team identifier
# train_data = pd.concat(preprocessed_datasets[:-1], ignore_index=True)
# aggregated_data = train_data.groupby("team identifier").mean().reset_index()
# 
# 
# 
# 
# # Create position embedding for the teams
# team_to_id = {team: idx for idx, team in enumerate(aggregated_data['team identifier'].unique(), start=1)}
# n_teams = len(team_to_id)
# position_embedding_dim = X.shape[1]  # Using the feature dimension as embedding dimension
# 
# # Create an input for raw features
# input_features = Input(shape=(X.shape[1],), name="input_features")
# 
# # Create an input for position
# input_position = Input(shape=(2,), name="input_position")
# 
# # Use an embedding layer for position embedding
# # Adjusting the input dimension for the embedding layer based on the maximum value in team_pairs
# max_team_id = np.max(team_pairs)
# position_embedding = Embedding(input_dim=max_team_id + 1, output_dim=position_embedding_dim, name="position_embedding")(input_position)
# 
# 
# # Take mean along the teams dimension
# position_embedding_mean = tf.keras.layers.Lambda(lambda x: tf.reduce_mean(x, axis=1))(position_embedding)
# 
# # Concatenate raw features and position embedding
# combined_input = Concatenate(axis=-1)([input_features, position_embedding_mean])
# 
# # Use a dense layer to combine them
# dense_combined = Dense(units=X.shape[1], activation='relu')(combined_input)
# 
# 
# # Model just for the input layer
# input_model = Model(inputs=[input_features, input_position], outputs=dense_combined)


In [57]:
# from keras.src.utils import to_categorical
# from sklearn.model_selection import train_test_split
# import tensorflow as tf
# from tensorflow.keras.layers import MultiHeadAttention, LayerNormalization, Add, Dropout
# from kerastuner import HyperModel, RandomSearch
# from creatingPairsForTrainingSetOne import trainingSetOne
# from tensorflow.keras.layers import Input, Embedding, Dense, Concatenate
# import numpy as np
# 
# combined_pairs_trainingSetOne, combined_labels_trainingSetOne, _, team_pairs = trainingSetOne()
# X= combined_pairs_trainingSetOne
# X = X.reshape(1140, -1)
# # Parameters for the transformer block
# d_model = X.shape[1]  # Dimension of the input vectors
# # num_heads = 4  # Number of attention heads
# # ff_dim = 256  # Hidden layer size in feed forward network inside transformer
# # dropout_rate = 0.1  # Dropout rate
# # num_transformer_blocks = 2  # Number of transformer blocks
# 
# # Define a Transformer block
# def transformer_block(inputs, num_heads, ff_dim, dropout_rate): 
#     # Add a sequence dimension
#     inputs = tf.expand_dims(inputs, 1)
# 
#     # Multi-head Self Attention
#     attention_output = MultiHeadAttention(num_heads=num_heads, key_dim=d_model)(inputs, inputs)
#     attention_output = Dropout(dropout_rate)(attention_output)
#     out1 = Add()([inputs, attention_output])
#     out1 = LayerNormalization(epsilon=1e-6)(out1)
# 
#     # Feed-forward network
#     ffn_output = Dense(ff_dim, activation='relu')(out1)
#     ffn_output = Dense(inputs.shape[-1])(ffn_output)  # Set the output dimension to match the input dimension
#     ffn_output = Dropout(dropout_rate)(ffn_output)
#     out2 = Add()([out1, ffn_output])
#     out2 = LayerNormalization(epsilon=1e-6)(out2)
# 
#     # Remove the sequence dimension
#     out2 = tf.squeeze(out2, 1)
# 
#     return out2
# 
# 
# class TransformerDualInputHyperModel(HyperModel):
#     def __init__(self, input_shape1, input_shape2):
#         self.input_shape1 = input_shape1
#         self.input_shape2 = input_shape2
# 
#     def build(self, hp):
#         input1 = tf.keras.Input(shape=self.input_shape1, name="input1")
#         input2 = tf.keras.Input(shape=self.input_shape2, name="input2")
#         
#         flattened_input1 = tf.keras.layers.Flatten()(input1)  # Flattening the player pairs
#         
#         merged_input = tf.keras.layers.Concatenate(axis=-1)([flattened_input1, input2])
#         
#         # Hyperparameters
#         # num_heads = 4  # Number of attention heads
#         # ff_dim = 256  # Hidden layer size in feed forward network inside transformer
#         num_heads = hp.Int('num_heads', 2, 4, step=2)
#         ff_dim = hp.Int('ff_dim', 128, 256, step=64)
#         dropout_rate = hp.Float('dropout_rate', 0.0, 0.2, step=0.1)
#         num_transformer_blocks = hp.Int('num_transformer_blocks', 1, 2)
# 
#         x = merged_input
#         for _ in range(num_transformer_blocks):
#             x = transformer_block(x, num_heads, ff_dim, dropout_rate)
# 
#         # Here, you can add your output layers based on the task, for example:
#         outputs = tf.keras.layers.Dense(30, activation='softmax')(x)  # Since there are 30 classes
# 
#         model = tf.keras.Model(inputs=[input1, input2], outputs=outputs)
#         model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
# 
#         return model
# 
# # Assume X_train is your input data
# encoded_labels = to_categorical(combined_labels_trainingSetOne, num_classes=30)
# team_pairs = np.array(team_pairs)
# 
# (train_pairs, val_pairs, train_teams, val_teams, train_labels, val_labels) = train_test_split(
#     combined_pairs_trainingSetOne,
#     team_pairs,  # Include team_pairs in the split
#     encoded_labels,
#     test_size=0.1,
#     random_state=42
# )
# 
# input_shape1 = (2, 51)  
# input_shape2 = (2,)  
# dual_hypermodel = TransformerDualInputHyperModel(input_shape1, input_shape2)
# 
# tuner = RandomSearch(
#     dual_hypermodel,
#     objective='val_accuracy',
#     max_trials=5,
#     executions_per_trial=1
# )
# 
# tuner.search(
#     [train_pairs, train_teams], train_labels,
#     validation_data=([val_pairs, val_teams], val_labels),
#     batch_size=30, epochs=20
# )
# 
# # 1. Get the best hyperparameters
# best_hp = tuner.get_best_hyperparameters()[0]
# 
# # 2. Print the best hyperparameters
# print("Best hyperparameters found:")
# for param, value in best_hp.values.items():
#     print(f"{param}: {value}")
# 


In [58]:
# from creatingPairsForTrainingSetOne import trainingSetOne
# from tensorflow.keras.utils import to_categorical
# 
# 
# best_hp = tuner.get_best_hyperparameters()[0]
# 
# 
# best_model = dual_hypermodel.build(best_hp)
# 
# 
# combined_pairs_trainingSetOne, combined_labels_trainingSetOne, _, team_pairs = trainingSetOne()
# encoded_labels = to_categorical(combined_labels_trainingSetOne, num_classes=30)
# print(combined_labels_trainingSetOne.shape)
# team_pairs = np.array(team_pairs)
# best_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
# best_model.fit([combined_pairs_trainingSetOne, team_pairs], encoded_labels, epochs=20, batch_size=30)
# 


In [59]:
# # Assume test_pairs are your input pairs for which you want predictions
# predictions = best_model.predict([combined_pairs_trainingSetOne, team_pairs])


In [60]:
# # Given the predictions
# # Calculate the average probability for each team
# average_probabilities = predictions.mean(axis=0)
# 
# # Team identifiers
# team_ids = [11, 8, 3, 14, 30, 22, 7, 5, 15, 27, 19, 12, 6, 2, 1, 16, 23, 29, 25, 9]
# 
# # Sort teams based on average probabilities
# sorted_teams = [team for _, team in sorted(zip(average_probabilities, team_ids), reverse=True)]
# sorted_teams_probs = sorted(zip(team_ids, average_probabilities), key=lambda x: x[1], reverse=True)
# 
# print(sorted_teams_probs)
# sorted_teams

In [61]:
# import pandas as pd
# 
# files = [
#     "../My_datasets/2012_13.xlsx",
#     "../My_datasets/2013_14.xlsx",
#     "../My_datasets/2014_15.xlsx",
#     "../My_datasets/2015_16.xlsx",
#     "../My_datasets/2016_17.xlsx",
#     "../My_datasets/2017_18.xlsx",
#     
# ]
# 
# season_standings = []
# 
# # Reading and processing each Excel file
# for file in files:
#     df = pd.read_excel(file)
#     # Extract the team identifier and its position based on index
#     standings_dict = {row['team identifier']: idx+1 for idx, row in df.iterrows()}
#     season_standings.append(standings_dict)
# 
# print(season_standings)


In [62]:
# import numpy as np
# 
# # Your provided data and model predictions
# predictions = best_model.predict([combined_pairs_trainingSetOne, team_pairs])
#  
# average_probabilities = predictions.mean(axis=0)
# print(average_probabilities)
# team_ids = [11, 8, 3, 14, 30, 22, 7, 5, 15, 27, 19, 12, 6, 2, 1, 16, 23, 29, 25, 9]
# 
# 
# season_weights = [1, 2, 3, 4, 5, 6]
# weighted_standings = []
# 
# for team_id in team_ids:
#     team_weighted_standing = sum(season_standings[season].get(team_id, len(team_ids)+1) * season_weights[season] for season in range(6))
#     weighted_standings.append(team_weighted_standing)
# 
# 
# 
# # Normalize the weighted standings
# max_weighted_value = len(team_ids) * sum(season_weights)
# normalized_standings = [(max_weighted_value - standing) / max_weighted_value for standing in weighted_standings]
# 
# # Weights for combining average probabilities and standings
# w1 = 0.7
# w2 = 0.3
# 
# # Adjust the average probabilities
# # Adjust only the probabilities corresponding to the 20 teams
# adjusted_probabilities = np.array(average_probabilities)
# adjusted_probabilities[:20] = w1 * average_probabilities[:20] + w2 * np.array(normalized_standings)
# 
# 
# 
# 
# # Sort teams based on adjusted probabilities
# sorted_teams = [team for _, team in sorted(zip(adjusted_probabilities, team_ids), reverse=True)]
# sorted_teams_probs = sorted(zip(team_ids, adjusted_probabilities), key=lambda x: x[1], reverse=True)
# 
# print(sorted_teams_probs)
# print(sorted_teams)


In [63]:
# import numpy as np
# 
# # Predictions from the model for pairs of teams
# predictions = best_model.predict([combined_pairs_trainingSetOne, team_pairs])
# 
# # Team identifiers
# team_ids = [11, 8, 3, 14, 30, 22, 7, 5, 15, 27, 19, 12, 6, 2, 1, 16, 23, 29, 25, 9]
# # print(team_pairs[:5])
# # Extract and aggregate scores for individual teams from pair predictions
# team_scores = {team_id: 0 for team_id in team_ids}
# for idx, pair in enumerate(team_pairs):
#     team1 = pair[0]
#     team2 = pair[1]
#     
#     try:
#         team_scores[team1] += predictions[idx]
#         print(team_scores[team1],":",predictions[idx])
#         team_scores[team2] += predictions[idx]
#     except KeyError:
#         print(f"Team identifier {team2} not found in team_scores dictionary.")
#         if team2 not in team_ids:
#             print(f"Team identifier {team2} is also not in the original team_ids list.")
# 
# # Normalize these aggregated scores to get average probabilities
# total_matches = len(combined_pairs_trainingSetOne) / 2  # Each team appears in half the pairs
# average_probabilities = {team_id: score / total_matches for team_id, score in team_scores.items()}
# 
# # Weights for standings
# season_weights = [1, 2, 3, 4, 5, 6]
# weighted_standings = []
# for team_id in team_ids:
#     team_weighted_standing = sum(season_standings[season].get(team_id, len(team_ids)+1) * season_weights[season] for season in range(6))
#     weighted_standings.append(team_weighted_standing)
# 
# # Normalize the weighted standings
# max_weighted_value = len(team_ids) * sum(season_weights)
# normalized_standings = [(max_weighted_value - standing) / max_weighted_value for standing in weighted_standings]
# 
# # Weights for combining average probabilities and standings
# w1 = 0.7
# w2 = 0.3
# 
# # Adjust the average probabilities
# adjusted_scores = {team_id: w1 * average_probabilities[team_id] + w2 * normalized_standings[team_ids.index(team_id)] for team_id in team_ids}
# for team_id, score in adjusted_scores.items():
#     print(f"Team ID: {team_id}, Score: {score}")
# 
# # Sort teams based on adjusted scores
# scalar_scores = {team_id: np.sum(score) for team_id, score in adjusted_scores.items()}
# 
# # Sort teams based on these scalar scores
# sorted_teams = [team for team, _ in sorted(scalar_scores.items(), key=lambda x: x[1], reverse=True)]
# sorted_teams_scores = sorted(scalar_scores.items(), key=lambda x: x[1], reverse=True)
# 
# print(sorted_teams_scores)
# print(sorted_teams)

In [1]:
#2018/19
from math import log2
with open('predicted_rankingsTrainingSetOne.txt', 'r') as f:
    predicted_rankings = [int(line.strip()) for line in f.readlines()]

actual_rankings = [11,
8,
3,
14,
30,
22,
7,
5,
15,
27,
19,
12,
6,
2,
1,
16,
23,
29,
25,
9

]
print(actual_rankings)







count_matches = 0
for a, p in zip(actual_rankings, predicted_rankings):
    if a == p:
        count_matches += 1

accuracy = count_matches / len(actual_rankings)
print(f'Accuracy: {accuracy * 100:.2f}%')

relevance_scores = {}
for rank, team in enumerate(actual_rankings):
    relevance_scores[team] = 19 - rank

def dcg_at_k(predicted_ranks, k):
    dcg = 0
    for idx, team in enumerate(predicted_ranks[:k]):
        dcg += (2**relevance_scores[team] - 1) / log2(idx + 2)
    return dcg


def ndcg_at_k(predicted_ranks, k):
    best_dcg = dcg_at_k(sorted(actual_rankings, key=lambda x: relevance_scores[x], reverse=True), k)
    actual_dcg = dcg_at_k(predicted_ranks, k)
    return actual_dcg / best_dcg
def compute_true_and_false_positives(predicted_rankings, actual_rankings, k):
    """Compute True Positives and False Positives up to rank k."""
    true_positives = 0
    for i in range(k):
        predicted_team = predicted_rankings[i]
        actual_rank = actual_rankings.index(predicted_team)
        if actual_rank < k:
            true_positives += 1
    false_positives = k - true_positives
    return true_positives, false_positives

def average_precision(predicted_rankings, actual_rankings, k=20):
    """Compute Average Precision up to rank k."""
    true_positives, false_positives = compute_true_and_false_positives(predicted_rankings, actual_rankings, k)
    return true_positives / (true_positives + false_positives)

def mean_average_precision(predicted_rankings, actual_rankings, k=20):
    """Compute Mean Average Precision up to rank k."""
    return sum(average_precision(predicted_rankings, actual_rankings, k=i) for i in range(1, k+1)) / k

# Sample rankings





# Computing mAP
k = 5
map_value = mean_average_precision(predicted_rankings, actual_rankings, k)
# print(f'DCG@{k}: {dcg_at_k(predicted_ranks_array, k):.2f}')
print(f'NDCG@{k}: {ndcg_at_k(predicted_rankings, k):.2f}')
print(f"Mean Average Precision (up to rank {k}): {map_value:.2f}")


k = 10
map_value = mean_average_precision(predicted_rankings, actual_rankings, k)
# print(f'DCG@{k}: {dcg_at_k(predicted_ranks_array, k):.2f}')
print(f'NDCG@{k}: {ndcg_at_k(predicted_rankings, k):.2f}')
print(f"Mean Average Precision (up to rank {k}): {map_value:.2f}")

k = 15
map_value = mean_average_precision(predicted_rankings, actual_rankings, k)
# print(f'DCG@{k}: {dcg_at_k(predicted_ranks_array, k):.2f}')
print(f'NDCG@{k}: {ndcg_at_k(predicted_rankings, k):.2f}')
print(f"Mean Average Precision (up to rank {k}): {map_value:.2f}")

# Computing mAP
k = 20
map_value = mean_average_precision(predicted_rankings, actual_rankings, k)
# print(f'DCG@{k}: {dcg_at_k(predicted_ranks_array, k):.2f}')
print(f'NDCG@{k}: {ndcg_at_k(predicted_rankings, k):.2f}')
print(f"Mean Average Precision (up to rank {k}): {map_value:.2f}")



[11, 8, 3, 14, 30, 22, 7, 5, 15, 27, 19, 12, 6, 2, 1, 16, 23, 29, 25, 9]
Accuracy: 35.00%
NDCG@5: 0.82
Mean Average Precision (up to rank 5): 0.74
NDCG@10: 0.93
Mean Average Precision (up to rank 10): 0.82
NDCG@15: 0.93
Mean Average Precision (up to rank 15): 0.84
NDCG@20: 0.93
Mean Average Precision (up to rank 20): 0.86


In [2]:
#2019/20
with open('predicted_rankingsTrainingSetTwo.txt', 'r') as f:
    predicted_rankings = [int(line.strip()) for line in f.readlines()]


print(predicted_rankings)
actual_rankings = [8,
11,
22,
3,
15,
14,
7,
30,
10,
1,
16,
5,
6,
12,
23,
27,
35,
2,
19,
28


]
print(actual_rankings)







count_matches = 0
for a, p in zip(actual_rankings, predicted_rankings):
    if a == p:
        count_matches += 1

accuracy = count_matches / len(actual_rankings)
print(f'Accuracy: {accuracy * 100:.2f}%')

relevance_scores = {}
for rank, team in enumerate(actual_rankings):
    relevance_scores[team] = 19 - rank

def dcg_at_k(predicted_ranks, k):
    dcg = 0
    for idx, team in enumerate(predicted_ranks[:k]):
        dcg += (2**relevance_scores[team] - 1) / log2(idx + 2)
    return dcg


def ndcg_at_k(predicted_ranks, k):
    best_dcg = dcg_at_k(sorted(actual_rankings, key=lambda x: relevance_scores[x], reverse=True), k)
    actual_dcg = dcg_at_k(predicted_ranks, k)
    return actual_dcg / best_dcg




# Computing mAP
k = 5
map_value = mean_average_precision(predicted_rankings, actual_rankings, k)
# print(f'DCG@{k}: {dcg_at_k(predicted_ranks_array, k):.2f}')
print(f'NDCG@{k}: {ndcg_at_k(predicted_rankings, k):.2f}')
print(f"Mean Average Precision (up to rank {k}): {map_value:.2f}")


k = 10
map_value = mean_average_precision(predicted_rankings, actual_rankings, k)
# print(f'DCG@{k}: {dcg_at_k(predicted_ranks_array, k):.2f}')
print(f'NDCG@{k}: {ndcg_at_k(predicted_rankings, k):.2f}')
print(f"Mean Average Precision (up to rank {k}): {map_value:.2f}")

k = 15
map_value = mean_average_precision(predicted_rankings, actual_rankings, k)
# print(f'DCG@{k}: {dcg_at_k(predicted_ranks_array, k):.2f}')
print(f'NDCG@{k}: {ndcg_at_k(predicted_rankings, k):.2f}')
print(f"Mean Average Precision (up to rank {k}): {map_value:.2f}")

# Computing mAP
k = 20
map_value = mean_average_precision(predicted_rankings, actual_rankings, k)
# print(f'DCG@{k}: {dcg_at_k(predicted_ranks_array, k):.2f}')
print(f'NDCG@{k}: {ndcg_at_k(predicted_rankings, k):.2f}')
print(f"Mean Average Precision (up to rank {k}): {map_value:.2f}")


[11, 14, 3, 8, 30, 22, 5, 15, 27, 16, 12, 2, 6, 19, 1, 7, 23, 35, 10, 28]
[8, 11, 22, 3, 15, 14, 7, 30, 10, 1, 16, 5, 6, 12, 23, 27, 35, 2, 19, 28]
Accuracy: 10.00%
NDCG@5: 0.67
Mean Average Precision (up to rank 5): 0.44
NDCG@10: 0.73
Mean Average Precision (up to rank 10): 0.61
NDCG@15: 0.73
Mean Average Precision (up to rank 15): 0.66
NDCG@20: 0.73
Mean Average Precision (up to rank 20): 0.73


In [3]:
#2020/21

with open('predicted_rankingsTrainingSetThree.txt', 'r') as f:
    predicted_rankings = [int(line.strip()) for line in f.readlines()]

actual_rankings =[11,
22,
8,
3,
15,
27,
14,
30,
18,
5,
35,
6,
7,
12,
16,
23,
1,
25,
21,
10

]
print(actual_rankings)







count_matches = 0
for a, p in zip(actual_rankings, predicted_rankings):
    if a == p:
        count_matches += 1

accuracy = count_matches / len(actual_rankings)
print(f'Accuracy: {accuracy * 100:.2f}%')

relevance_scores = {}
for rank, team in enumerate(actual_rankings):
    relevance_scores[team] = 19 - rank

def dcg_at_k(predicted_ranks, k):
    dcg = 0
    for idx, team in enumerate(predicted_ranks[:k]):
        dcg += (2**relevance_scores[team] - 1) / log2(idx + 2)
    return dcg


def ndcg_at_k(predicted_ranks, k):
    best_dcg = dcg_at_k(sorted(actual_rankings, key=lambda x: relevance_scores[x], reverse=True), k)
    actual_dcg = dcg_at_k(predicted_ranks, k)
    return actual_dcg / best_dcg




# Computing mAP
k = 5
map_value = mean_average_precision(predicted_rankings, actual_rankings, k)
# print(f'DCG@{k}: {dcg_at_k(predicted_ranks_array, k):.2f}')
print(f'NDCG@{k}: {ndcg_at_k(predicted_rankings, k):.2f}')
print(f"Mean Average Precision (up to rank {k}): {map_value:.2f}")


k = 10
map_value = mean_average_precision(predicted_rankings, actual_rankings, k)
# print(f'DCG@{k}: {dcg_at_k(predicted_ranks_array, k):.2f}')
print(f'NDCG@{k}: {ndcg_at_k(predicted_rankings, k):.2f}')
print(f"Mean Average Precision (up to rank {k}): {map_value:.2f}")

k = 15
map_value = mean_average_precision(predicted_rankings, actual_rankings, k)
# print(f'DCG@{k}: {dcg_at_k(predicted_ranks_array, k):.2f}')
print(f'NDCG@{k}: {ndcg_at_k(predicted_rankings, k):.2f}')
print(f"Mean Average Precision (up to rank {k}): {map_value:.2f}")

# Computing mAP
k = 20
map_value = mean_average_precision(predicted_rankings, actual_rankings, k)
# print(f'DCG@{k}: {dcg_at_k(predicted_ranks_array, k):.2f}')
print(f'NDCG@{k}: {ndcg_at_k(predicted_rankings, k):.2f}')
print(f"Mean Average Precision (up to rank {k}): {map_value:.2f}")

[11, 22, 8, 3, 15, 27, 14, 30, 18, 5, 35, 6, 7, 12, 16, 23, 1, 25, 21, 10]
Accuracy: 15.00%
NDCG@5: 0.94
Mean Average Precision (up to rank 5): 0.74
NDCG@10: 0.94
Mean Average Precision (up to rank 10): 0.77
NDCG@15: 0.94
Mean Average Precision (up to rank 15): 0.79
NDCG@20: 0.94
Mean Average Precision (up to rank 20): 0.82


In [8]:
#2021/22
# Load the predicted rankings
with open('predicted_rankingsTrainingSetFour.txt', 'r') as f:
    predicted_rankings = [int(line.strip()) for line in f.readlines()]

# Print the loaded predicted rankings
print(predicted_rankings)

# Define the actual rankings
actual_rankings = [11,
8,
3,
14,
30,
22,
27,
15,
23,
7,
6,
12,
33,
35,
16,
5,
18,
1,
19,
28
]
print(actual_rankings)

# Compute and print the accuracy
count_matches = 0
for a, p in zip(actual_rankings, predicted_ranks_array):
    if a == p:
        count_matches += 1
accuracy = count_matches / len(actual_rankings)
print(f'Accuracy: {accuracy * 100:.2f}%')

# Define relevance scores for DCG calculation
relevance_scores = {}
for rank, team in enumerate(actual_rankings):
    relevance_scores[team] = 19 - rank

def dcg_at_k(predicted_ranks, k):
    dcg = 0
    for idx, team in enumerate(predicted_ranks[:k]):
        dcg += (2**relevance_scores[team] - 1) / log2(idx + 2)
    return dcg

def ndcg_at_k(predicted_ranks, k):
    best_dcg = dcg_at_k(sorted(actual_rankings, key=lambda x: relevance_scores[x], reverse=True), k)
    actual_dcg = dcg_at_k(predicted_ranks, k)
    return actual_dcg / best_dcg

def compute_relevance(predicted_rankings, actual_rankings):
    """Compute continuous relevance values based on rank difference."""
    relevance = []
    for predicted_team in predicted_rankings:
        predicted_rank = predicted_rankings.index(predicted_team)
        actual_rank = actual_rankings.index(predicted_team)
        # The relevance is inversely proportional to rank difference
        rel = 1 / (abs(predicted_rank - actual_rank) + 1)
        relevance.append(rel)
    return relevance


def compute_true_and_false_positives(predicted_rankings, actual_rankings, k):
    """Compute True Positives and False Positives up to rank k."""
    true_positives = 0
    for i in range(k):
        predicted_team = predicted_rankings[i]
        actual_rank = actual_rankings.index(predicted_team)
        if actual_rank < k:
            true_positives += 1
    false_positives = k - true_positives
    return true_positives, false_positives

def average_precision(predicted_rankings, actual_rankings, k=20):
    """Compute Average Precision up to rank k."""
    true_positives, false_positives = compute_true_and_false_positives(predicted_rankings, actual_rankings, k)
    return true_positives / (true_positives + false_positives)

def mean_average_precision(predicted_rankings, actual_rankings, k=20):
    """Compute Mean Average Precision up to rank k."""
    return sum(average_precision(predicted_rankings, actual_rankings, k=i) for i in range(1, k+1)) / k





# Computing mAP
k = 5
map_value = mean_average_precision(predicted_rankings, actual_rankings, k)
# print(f'DCG@{k}: {dcg_at_k(predicted_ranks_array, k):.2f}')
print(f'NDCG@{k}: {ndcg_at_k(predicted_rankings, k):.2f}')
print(f"Mean Average Precision (up to rank {k}): {map_value:.2f}")


k = 10
map_value = mean_average_precision(predicted_rankings, actual_rankings, k)
# print(f'DCG@{k}: {dcg_at_k(predicted_ranks_array, k):.2f}')
print(f'NDCG@{k}: {ndcg_at_k(predicted_rankings, k):.2f}')
print(f"Mean Average Precision (up to rank {k}): {map_value:.2f}")

k = 15
map_value = mean_average_precision(predicted_rankings, actual_rankings, k)
# print(f'DCG@{k}: {dcg_at_k(predicted_ranks_array, k):.2f}')
print(f'NDCG@{k}: {ndcg_at_k(predicted_rankings, k):.2f}')
print(f"Mean Average Precision (up to rank {k}): {map_value:.2f}")

# Computing mAP
k = 20
map_value = mean_average_precision(predicted_rankings, actual_rankings, k)
# print(f'DCG@{k}: {dcg_at_k(predicted_ranks_array, k):.2f}')
print(f'NDCG@{k}: {ndcg_at_k(predicted_rankings, k):.2f}')
print(f"Mean Average Precision (up to rank {k}): {map_value:.2f}")
 

[11, 8, 22, 3, 14, 30, 15, 5, 27, 16, 12, 6, 1, 7, 19, 35, 23, 18, 33, 28]
[11, 8, 3, 14, 30, 22, 27, 15, 23, 7, 6, 12, 33, 35, 16, 5, 18, 1, 19, 28]
Accuracy: 15.00%
NDCG@5: 0.98
Mean Average Precision (up to rank 5): 0.84
NDCG@10: 0.99
Mean Average Precision (up to rank 10): 0.86
NDCG@15: 0.99
Mean Average Precision (up to rank 15): 0.84
NDCG@20: 0.99
Mean Average Precision (up to rank 20): 0.86


In [9]:
#2022/23

# Load the predicted rankings
with open('predicted_rankingsTrainingSetFive.txt', 'r') as f:
    predicted_rankings = [int(line.strip()) for line in f.readlines()]

# Print the loaded predicted rankings
print(predicted_rankings)

# Define the actual rankings
actual_rankings =  [11,
30,
22,
6,
8,
23,
35,
14,
33,
25,
12,
3,
7,
27,
2,
32,
5,
15,
18,
16


]
print(actual_rankings)

# Compute and print the accuracy
count_matches = 0
for a, p in zip(actual_rankings, predicted_ranks_array):
    if a == p:
        count_matches += 1
accuracy = count_matches / len(actual_rankings)
print(f'Accuracy: {accuracy * 100:.2f}%')

# Define relevance scores for DCG calculation
relevance_scores = {}
for rank, team in enumerate(actual_rankings):
    relevance_scores[team] = 19 - rank

def dcg_at_k(predicted_ranks, k):
    dcg = 0
    for idx, team in enumerate(predicted_ranks[:k]):
        dcg += (2**relevance_scores[team] - 1) / log2(idx + 2)
    return dcg

def ndcg_at_k(predicted_ranks, k):
    best_dcg = dcg_at_k(sorted(actual_rankings, key=lambda x: relevance_scores[x], reverse=True), k)
    actual_dcg = dcg_at_k(predicted_ranks, k)
    return actual_dcg / best_dcg

def compute_relevance(predicted_rankings, actual_rankings):
    """Compute continuous relevance values based on rank difference."""
    relevance = []
    for predicted_team in predicted_rankings:
        predicted_rank = predicted_rankings.index(predicted_team)
        actual_rank = actual_rankings.index(predicted_team)
        # The relevance is inversely proportional to rank difference
        rel = 1 / (abs(predicted_rank - actual_rank) + 1)
        relevance.append(rel)
    return relevance


def compute_true_and_false_positives(predicted_rankings, actual_rankings, k):
    """Compute True Positives and False Positives up to rank k."""
    true_positives = 0
    for i in range(k):
        predicted_team = predicted_rankings[i]
        actual_rank = actual_rankings.index(predicted_team)
        if actual_rank < k:
            true_positives += 1
    false_positives = k - true_positives
    return true_positives, false_positives

def average_precision(predicted_rankings, actual_rankings, k=20):
    """Compute Average Precision up to rank k."""
    true_positives, false_positives = compute_true_and_false_positives(predicted_rankings, actual_rankings, k)
    return true_positives / (true_positives + false_positives)

def mean_average_precision(predicted_rankings, actual_rankings, k=20):
    """Compute Mean Average Precision up to rank k."""
    return sum(average_precision(predicted_rankings, actual_rankings, k=i) for i in range(1, k+1)) / k

# Sample rankings




def compute_true_and_false_positives(predicted_rankings, actual_rankings, k):
    """Compute True Positives and False Positives up to rank k."""
    true_positives = 0
    for i in range(k):
        predicted_team = predicted_rankings[i]
        actual_rank = actual_rankings.index(predicted_team)
        if actual_rank < k:
            true_positives += 1
    false_positives = k - true_positives
    return true_positives, false_positives

def average_precision(predicted_rankings, actual_rankings, k=20):
    """Compute Average Precision up to rank k."""
    true_positives, false_positives = compute_true_and_false_positives(predicted_rankings, actual_rankings, k)
    return true_positives / (true_positives + false_positives)

def mean_average_precision(predicted_rankings, actual_rankings, k=20):
    """Compute Mean Average Precision up to rank k."""
    return sum(average_precision(predicted_rankings, actual_rankings, k=i) for i in range(1, k+1)) / k





# Computing mAP
k = 5
map_value = mean_average_precision(predicted_rankings, actual_rankings, k)
# print(f'DCG@{k}: {dcg_at_k(predicted_ranks_array, k):.2f}')
print(f'NDCG@{k}: {ndcg_at_k(predicted_rankings, k):.2f}')
print(f"Mean Average Precision (up to rank {k}): {map_value:.2f}")


k = 10
map_value = mean_average_precision(predicted_rankings, actual_rankings, k)
# print(f'DCG@{k}: {dcg_at_k(predicted_ranks_array, k):.2f}')
print(f'NDCG@{k}: {ndcg_at_k(predicted_rankings, k):.2f}')
print(f"Mean Average Precision (up to rank {k}): {map_value:.2f}")

k = 15
map_value = mean_average_precision(predicted_rankings, actual_rankings, k)
# print(f'DCG@{k}: {dcg_at_k(predicted_ranks_array, k):.2f}')
print(f'NDCG@{k}: {ndcg_at_k(predicted_rankings, k):.2f}')
print(f"Mean Average Precision (up to rank {k}): {map_value:.2f}")

# Computing mAP
k = 20
map_value = mean_average_precision(predicted_rankings, actual_rankings, k)
# print(f'DCG@{k}: {dcg_at_k(predicted_ranks_array, k):.2f}')
print(f'NDCG@{k}: {ndcg_at_k(predicted_rankings, k):.2f}')
print(f"Mean Average Precision (up to rank {k}): {map_value:.2f}")
 
 

[11, 14, 3, 8, 30, 22, 5, 15, 27, 16, 12, 6, 2, 7, 23, 35, 25, 18, 33, 32]
[11, 30, 22, 6, 8, 23, 35, 14, 33, 25, 12, 3, 7, 27, 2, 32, 5, 15, 18, 16]
Accuracy: 15.00%
NDCG@5: 0.81
Mean Average Precision (up to rank 5): 0.54
NDCG@10: 0.85
Mean Average Precision (up to rank 10): 0.56
NDCG@15: 0.88
Mean Average Precision (up to rank 15): 0.60
NDCG@20: 0.88
Mean Average Precision (up to rank 20): 0.67
